In [58]:
import tensorflow
import PIL
from PIL import Image
from matplotlib.pyplot import imshow

import os

import numpy as np

from scipy.spatial.distance import cosine

In [59]:
!pip install keras_applications
!pip install git+https://github.com/rcmalli/keras-vggface.git

You are using pip version 18.1, however version 20.2.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.

  Cloning https://github.com/rcmalli/keras-vggface.git to c:\users\rongw\appdata\local\temp\pip-req-build-9luk465y
  Running setup.py bdist_wheel for keras-vggface: started
  Running setup.py bdist_wheel for keras-vggface: finished with status 'done'
  Stored in directory: C:\Users\rongw\AppData\Local\Temp\pip-ephem-wheel-cache-5jgypnhk\wheels\36\07\46\06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [60]:
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

In [61]:
image_dir = 'faces/aligned/rong/'
other_dir = 'faces/aligned/notrong/'

In [62]:
model = VGGFace(include_top=False, input_shape=(244,244,3), pooling='avg')

print(model.inputs)

[<tf.Tensor 'input_12:0' shape=(None, 244, 244, 3) dtype=float32>]


In [83]:
face_embeddings = []
face_embeddings_not = []

def get_embedding(image_path, model, image_size):
  img = Image.open(image_path)
  img = img.resize(image_size)
  pix = np.array(img)
  pix = pix.astype('float32')
  pix = np.expand_dims(pix, axis=0)

  face = preprocess_input(pix, version=2)
  return model.predict(face)

size = (244, 244)
for file in os.listdir(image_dir):
  face_embeddings.append(get_embedding(image_dir + file, model, size))

for file in os.listdir(other_dir):
  face_embeddings_not.append(get_embedding(other_dir + file, model, size))

In [91]:
max = 0
for i in range(len(face_embeddings)):
  for j in range(i + 1, len(face_embeddings)):
    dist = cosine(face_embeddings[i], face_embeddings[j])
    if(max < dist): 
      max = dist

print('highest distance bt photos of me: ', max)

min_index = 0
min_dist = 1
for i in range(len(face_embeddings)):
  for j in range(len(face_embeddings_not)):
    dist = (cosine(face_embeddings[i], face_embeddings_not[j]))
    if(dist < min_dist):
      min_dist = dist
      min_index = j

print('most similar image dist value of not me: ' , min_dist)
print('image index: ', min_index)

highest distance bt photos of me:  0.24043625593185425
most similar image dist value of not me:  0.3000403642654419
image index:  3


In [65]:
vgg = VGGFace()

for file in os.listdir(image_dir):
  img = Image.open(image_dir + file)
  img = img.resize(size)
  pix = np.array(img)
  pix = pix.astype('float32')
  pix = np.expand_dims(pix, axis=0)

  pix = preprocess_input(pix, version=2)
  res = vgg.predict(pix)
  results = decode_predictions(res)

  print(results)


[[["b'Jake_Cherry'", 0.034130726], ["b'Sung_Kang'", 0.03310631], ["b'Sophie_Okonedo'", 0.0274092], ["b'Cierra_Ramirez'", 0.026795443], ["b'Britne_Oldford'", 0.018653762]]]
[[["b'Rico_Rodriguez'", 0.10262949], ["b'Sophie_Okonedo'", 0.07488068], ["b'Shonda_Rhimes'", 0.047370546], ["b'Raini_Rodriguez'", 0.044577476], ["b'David_Mamet'", 0.038508985]]]
[[["b'Toni_Trucks'", 0.13743211], ["b'Della_Reese'", 0.054906514], ["b'Reece_Thompson'", 0.023407605], ["b'Harry_Styles'", 0.019214308], ["b'Efren_Ramirez'", 0.017902216]]]
[[["b'Della_Reese'", 0.18197499], ["b'Toni_Trucks'", 0.038251713], ["b'Reece_Thompson'", 0.017681155], ["b'Loretta_Devine'", 0.01573061], ["b'Efren_Ramirez'", 0.015394212]]]
[[["b'Kali_Hawk'", 0.054283492], ["b'Reece_Thompson'", 0.028296957], ["b'Efren_Ramirez'", 0.021056468], ["b'Della_Reese'", 0.018407248], ["b'Dr._Dre'", 0.0157496]]]
[[["b'Kali_Hawk'", 0.04814082], ["b'Andy_Milonakis'", 0.048020273], ["b'Jimmy_Bennett'", 0.04611696], ["b'Kel_Mitchell'", 0.04074806], ["b

In [66]:
rong = Image.open('faces/aligned/rong/face_0.jpg')
rong1 = Image.open('faces/aligned/rong/face_1.jpg')

rong = rong.resize(size)
pix_rong = np.array(rong)
pix_rong = pix_rong.astype('float32')
pix_rong = np.expand_dims(pix_rong, axis=0)
pix_rong = preprocess_input(pix_rong, version=2)
res = model.predict(pix_rong)

rong1 = rong1.resize(size)
rong1 = rong1.resize(size)
pix_rong1 = np.array(rong1)
pix_rong1 = pix_rong1.astype('float32')
pix_rong1 = np.expand_dims(pix_rong1, axis=0)
pix_rong1 = preprocess_input(pix_rong1, version=2)
res1 = model.predict(pix_rong1)

notrong = Image.open('faces/aligned/notrong/face_0.jpg')
notrong = notrong.resize(size)
pix_not = np.array(notrong)
pix_not = pix_not.astype('float32')
pix_not = np.expand_dims(pix_not, axis=0)
pix_not = preprocess_input(pix_not)
res2 = model.predict(pix_not)

print(cosine(res, res1))
print(cosine(res, res2))
print(cosine(res1, res2))


0.20365482568740845
0.5448296070098877
0.5288482904434204
